In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# === ЗАГРУЗКА ДАННЫХ ===
train_df = pd.read_csv('/kaggle/input/machine-translation-ioai/train.csv')
test_df = pd.read_csv('/kaggle/input/machine-translation-ioai/test.csv')

In [3]:
# Подготовка данных
input_texts = train_df["data"].astype(str).tolist()
target_texts = train_df["label"].astype(str).tolist()
target_texts = ["<start> " + text + " <end>" for text in target_texts]

In [4]:
# Входная токенизация
input_tokenizer = Tokenizer(char_level=True, filters="")
input_tokenizer.fit_on_texts(input_texts)
input_sequences = input_tokenizer.texts_to_sequences(input_texts)
input_padded = pad_sequences(input_sequences, padding="post")

# Выходная токенизация
target_tokenizer = Tokenizer(char_level=True, filters="")
target_tokenizer.fit_on_texts(target_texts)
target_tokenizer.word_index["<start>"] = len(target_tokenizer.word_index) + 1
target_tokenizer.word_index["<end>"] = len(target_tokenizer.word_index) + 1
target_sequences = target_tokenizer.texts_to_sequences(target_texts)
target_padded = pad_sequences(target_sequences, padding="post")

# Размерности словарей
num_encoder_tokens = len(input_tokenizer.word_index) + 1
num_decoder_tokens = len(target_tokenizer.word_index) + 1
max_encoder_seq_length = input_padded.shape[1]
max_decoder_seq_length = target_padded.shape[1]


In [5]:
# === СОЗДАНИЕ DATASET ===
class TranslationDataset(Dataset):
    def __init__(self, encoder_input, decoder_input, decoder_target):
        self.encoder_input = torch.tensor(encoder_input, dtype=torch.long)
        self.decoder_input = torch.tensor(decoder_input, dtype=torch.long)
        self.decoder_target = torch.tensor(decoder_target, dtype=torch.long)

    def __len__(self):
        return len(self.encoder_input)

    def __getitem__(self, idx):
        return self.encoder_input[idx], self.decoder_input[idx], self.decoder_target[idx]


In [6]:
# Разделяем на train/validation (90%/10%)
decoder_input_data = target_padded[:, :-1]
decoder_target_data = target_padded[:, 1:]
full_dataset = TranslationDataset(input_padded, decoder_input_data, decoder_target_data)

train_size = int(0.99 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [7]:
# === СОЗДАНИЕ МОДЕЛИ ===
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hidden_dim):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim, padding_idx=0)
        self.lstm = nn.LSTM(emb_dim, hidden_dim, batch_first=True)

    def forward(self, x):
        embedded = self.embedding(x)
        outputs, (hidden, cell) = self.lstm(embedded)
        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hidden_dim):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(output_dim, emb_dim, padding_idx=0)
        self.lstm = nn.LSTM(emb_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, hidden, cell):
        embedded = self.embedding(x.unsqueeze(1))
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        prediction = self.fc(output.squeeze(1))
        return prediction, hidden, cell

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, trg):
        batch_size, trg_len = trg.shape
        outputs = torch.zeros(batch_size, trg_len, num_decoder_tokens).to(device)

        hidden, cell = self.encoder(src)
        input = trg[:, 0]

        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[:, t] = output
            input = output.argmax(1)

        return outputs


In [8]:
emb_dim = 512  # Размерность эмбеддингов
hidden_dim = 1024  # Размер скрытых состояний

In [9]:
# === ОБУЧЕНИЕ ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(num_encoder_tokens, emb_dim, hidden_dim).to(device)
decoder = Decoder(num_decoder_tokens, emb_dim, hidden_dim).to(device)
model = Seq2Seq(encoder, decoder).to(device)

criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [10]:
num_epochs = 1000

In [11]:
# Инициализируем переменные для отслеживания лучшей модели
best_val_loss = float('inf')  # Начальная наилучшая валид. потеря (очень большая)
best_model_state = None  # Для хранения состояния лучшей модели

for epoch in range(num_epochs):  
    # === ОБУЧЕНИЕ ===
    model.train()
    train_loss = 0

    for enc_input, dec_input, dec_target in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Training]"):
        enc_input, dec_input, dec_target = enc_input.to(device), dec_input.to(device), dec_target.to(device)
        
        predictions = model(enc_input, dec_input)
        loss = criterion(predictions.reshape(-1, num_decoder_tokens), dec_target.reshape(-1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader)
    
    # === ВАЛИДАЦИЯ ===
    model.eval()
    val_loss = 0

    with torch.no_grad():
        for enc_input, dec_input, dec_target in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Validation]"):
            enc_input, dec_input, dec_target = enc_input.to(device), dec_input.to(device), dec_target.to(device)

            predictions = model(enc_input, dec_input)
            loss = criterion(predictions.reshape(-1, num_decoder_tokens), dec_target.reshape(-1))
            val_loss += loss.item()

    val_loss /= len(val_loader)

    print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f} | Val Loss = {val_loss:.4f}")

    # Сохраняем модель, если валидационная потеря улучшилась
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_state = model.state_dict()  # Сохраняем состояние модели

# После завершения обучения загружаем лучшие веса модели
model.load_state_dict(best_model_state)
print("Лучшие веса модели сохранены!")


Epoch 1/1000 [Validation]: 100%|██████████| 1/1 [00:00<00:00, 46.55it/s]


Epoch 1: Train Loss = 2.3801 | Val Loss = 1.5633


Epoch 2/1000 [Validation]: 100%|██████████| 1/1 [00:00<00:00, 108.55it/s]


Epoch 2: Train Loss = 1.3468 | Val Loss = 1.1008


Epoch 3/1000 [Validation]: 100%|██████████| 1/1 [00:00<00:00, 102.47it/s]


...

Epoch 998: Train Loss = 0.1382 | Val Loss = 0.1382


Epoch 999/1000 [Validation]: 100%|██████████| 1/1 [00:00<00:00, 114.17it/s]


Epoch 999: Train Loss = 0.1382 | Val Loss = 0.1382


Epoch 1000/1000 [Validation]: 100%|██████████| 1/1 [00:00<00:00, 110.89it/s]

Epoch 1000: Train Loss = 0.1382 | Val Loss = 0.1382
Лучшие веса модели сохранены!


In [12]:
# === ПРЕДСКАЗАНИЕ ===
def translate_batch(input_seqs):
    model.eval()
    input_seqs = torch.tensor(input_seqs, dtype=torch.long).to(device)
    hidden, cell = encoder(input_seqs)
    target_seqs = torch.full((input_seqs.shape[0], 1), target_tokenizer.word_index["<start>"], dtype=torch.long).to(device)

    decoded_sentences = [""] * len(input_seqs)
    stop_condition = [False] * len(input_seqs)

    for _ in range(max_decoder_seq_length):
        output, hidden, cell = decoder(target_seqs[:, -1], hidden, cell)
        sampled_indices = output.argmax(1)

        for i in range(len(input_seqs)):
            if not stop_condition[i]:
                sampled_char = target_tokenizer.index_word.get(sampled_indices[i].item(), "")
                if sampled_char == "<end>":
                    stop_condition[i] = True
                else:
                    decoded_sentences[i] += sampled_char
                    target_seqs = torch.cat((target_seqs, sampled_indices.unsqueeze(1)), dim=1)

        if all(stop_condition):
            break

    return [sentence.strip() for sentence in decoded_sentences]

In [13]:
# === ОТРИСОВКА ПРЕДСКАЗАНИЙ ===
test_texts = test_df["data"].astype(str).tolist()
test_sequences = input_tokenizer.texts_to_sequences(test_texts)
test_padded = pad_sequences(test_sequences, maxlen=max_encoder_seq_length, padding="post")


In [14]:
batch_size = 64


In [15]:
predictions = []
for i in tqdm(range(0, len(test_padded), batch_size), desc="Predicting"):
    batch_input = test_padded[i:i + batch_size]
    decoded_sentences = translate_batch(batch_input)
    predictions.extend(decoded_sentences)

Predicting: 100%|██████████| 74/74 [00:05<00:00, 14.75it/s]


In [16]:
submission = pd.DataFrame({"id": test_df["id"], "label": predictions})
submission['label'] = submission['label'].apply(lambda x: x.split('art>')[1].split('<end')[0].strip())


In [17]:
submission

,id,label
0,0,24-01-2007
1,1,06-03-2049
2,2,10-05-2077
3,3,27-06-2049
4,4,08-01-2077
...,...,...
4671,4671,15-01-2049
4672,4672,09-05-2049
4673,4673,18-02-2007
4674,4674,14-12-2049


In [18]:
submission.to_csv("submission.csv", index=False)